# Mobile Get Input Notebook - Phase 1: Data Preparation

**Phase 1**: Download and cache land cover data from Sentinel Hub.

This notebook downloads the land cover GeoTIFF for the transmitter location and caches it locally.

## Prerequisites
- Run **Phase 0** first (phase0_setup.ipynb) to set up environment, CONFIG, and transmitter

## Workflow
1. **Import Phase 0 outputs**: CONFIG, transmitter object, credentials, path setup
2. **Resolve Sentinel Hub credentials**: Get API access
3. **Fetch land cover**: Download GeoTIFF from Sentinel Hub API
4. **Cache locally**: Save to `data/intermediate/api_data/`

## Output
- Cached GeoTIFF file in `data/intermediate/api_data/lcm10_<lat>_<lon>_<year>.tif`
- Ready for Phase 2-3 (batch processing)

## Setup: Import from Phase 0

This cell imports all the setup from Phase 0 (CONFIG, paths, transmitter, etc.).

In [13]:
# Import everything from Phase 0 using %run magic
%run phase0_setup.ipynb

print("\n✓ Phase 0 setup imported successfully")
print(f"  CONFIG: {list(CONFIG.keys())}")
print(f"  Transmitter: {tx}")

✓ All imports successful
Project root: /Users/oz/Documents/mst_gis
Profiles dir: /Users/oz/Documents/mst_gis/data/input/profiles
API data dir: /Users/oz/Documents/mst_gis/data/intermediate/api_data
Workflow dir: /Users/oz/Documents/mst_gis/data/intermediate/workflow
Reference dir: /Users/oz/Documents/mst_gis/data/input/reference
Transmitter: (9.345, -13.40694)
Azimuths: 36 | Profile points: 366
Frequency: 0.9 GHz | Polarization: 1

✓ Transmitter created:
  Transmitter(tx_id='TX_0001', lon=-13.40694, lat=9.345, htg=57, f=0.9, pol=1, p=50, hrg=10)

Seeding elevation data...
make: Nothing to be done for `download'.
make: Nothing to be done for `all'.
✓ Elevation data ready (0.05s)
  Cache location: /Users/oz/Library/Caches/elevation

PHASE 0 COMPLETE: Setup ready for subsequent phases

Next steps:
  1. Run Phase 1 to download/cache land cover data
  2. Run Phase 2 to generate all receiver points
  3. Run Phase 3 to extract elevation and land cover
  4. Run Phase 4 to format and export pro

## Additional Imports

Import modules needed by Phase 1 helper functions.

In [14]:
# These are imported by Phase 0 but we import them here explicitly for clarity
import requests
from rasterio.io import MemoryFile

# Verify Phase 0 imports are available
assert 'CONFIG' in locals(), 'CONFIG not defined - run Phase 0 first'
assert 'tx' in locals(), 'tx not defined - run Phase 0 first'
assert 'api_data_dir' in locals(), 'api_data_dir not defined - run Phase 0 first'
assert 'TOKEN_URL' in locals(), 'TOKEN_URL not defined - run Phase 0 first'
assert 'PROCESS_URL' in locals(), 'PROCESS_URL not defined - run Phase 0 first'
assert 'COLLECTION_ID' in locals(), 'COLLECTION_ID not defined - run Phase 0 first'
assert 'SH_CLIENT_ID' in locals(), 'SH_CLIENT_ID not defined - run Phase 0 first'
assert 'SH_CLIENT_SECRET' in locals(), 'SH_CLIENT_SECRET not defined - run Phase 0 first'

print("✓ All required imports and Phase 0 outputs available")

✓ All required imports and Phase 0 outputs available


## Helper Functions

Define reusable functions for Sentinel Hub API interaction and credential handling.

In [15]:
def resolve_credentials(verbose: bool = True, fallback_id=None, fallback_secret=None):
    """Get Sentinel Hub credentials from environment or fallback."""
    env_id = os.environ.get("SH_CLIENT_ID", "").strip()
    env_secret = os.environ.get("SH_CLIENT_SECRET", "").strip()

    if env_id and env_secret:
        if verbose:
            print("  ✓ Found credentials in environment variables")
        return env_id, env_secret

    const_id = (fallback_id or SH_CLIENT_ID or "").strip()
    const_secret = (fallback_secret or SH_CLIENT_SECRET or "").strip()

    if not const_id or not const_secret or "REPLACE_ME" in str(const_id) or "REPLACE_ME" in str(const_secret):
        raise RuntimeError("Credentials not found. Set SH_CLIENT_ID and SH_CLIENT_SECRET.")

    if verbose:
        print("  ✓ Found credentials in config file")
    return const_id, const_secret


def meters_to_deg(lat, meters):
    dlat = meters / 111_320.0
    dlon = meters / (111_320.0 * math.cos(math.radians(lat)))
    return dlat, dlon


def get_token(client_id, client_secret):
    r = requests.post(TOKEN_URL, data={
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
    }, timeout=30)
    r.raise_for_status()
    return r.json().get("access_token")


def landcover_at_point(client_id, client_secret, lat, lon, token_url, process_url, collection_id, year=2020, buffer_m=1000, chip_px=32, save_path=None):
    token = get_token(client_id, client_secret)
    dlat, dlon = meters_to_deg(lat, buffer_m)
    bbox = [lon - dlon, lat - dlat, lon + dlon, lat + dlat]

    evalscript = """//VERSION=3
function setup() { return {input: [\"LCM10\"], output: {bands: 1, sampleType: \"UINT8\"}};}function evaluatePixel(s) { return [s.LCM10];}"""

    body = {
        "input": {"bounds": {"bbox": bbox, "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"}},
                   "data": [{"type": f"byoc-{collection_id}", "dataFilter": {"timeRange": {"from": f"{year}-01-01T00:00:00Z", "to": f"{year}-12-31T23:59:59Z"}}}]},
        "output": {"width": chip_px, "height": chip_px, "responses": [{"identifier": "default", "format": {"type": "image/tiff"}}]},
        "evalscript": evalscript,
    }

    r = requests.post(process_url, json=body, headers={
        "Authorization": f"Bearer {token}", "Content-Type": "application/json", "Accept": "image/tiff"},
        timeout=120)
    r.raise_for_status()

    if save_path:
        with open(save_path, "wb") as f:
            f.write(r.content)

    with MemoryFile(r.content) as memfile:
        with memfile.open() as ds:
            arr = ds.read(1)

    center_code = int(arr[arr.shape[0] // 2, arr.shape[1] // 2])
    return center_code, arr


print("✓ Helper functions defined")

✓ Helper functions defined


## Fetch and Cache Land Cover

In [16]:
print("\n" + "="*60)
print("PHASE 1: DATA PREPARATION - Fetch Land Cover")
print("="*60)

# Resolve credentials
print("\nResolving Sentinel Hub credentials...")
client_id, client_secret = resolve_credentials(
    fallback_id=SH_CLIENT_ID,
    fallback_secret=SH_CLIENT_SECRET,
    verbose=True
)

# Extract parameters from CONFIG
lat = CONFIG['TRANSMITTER']['latitude']
lon = CONFIG['TRANSMITTER']['longitude']
buffer_m = CONFIG['SENTINEL_HUB']['buffer_m']
chip_px = CONFIG['SENTINEL_HUB']['chip_px']
year = CONFIG['SENTINEL_HUB']['year']

# Define output path
out_tif = api_data_dir / f"lcm10_{lat}_{lon}_{year}_buf{buffer_m}m_{chip_px}px.tif"

print(f"\nFetching land cover for transmitter:")
print(f"  Location: ({lat}, {lon})")
print(f"  Buffer: {buffer_m}m | Chip: {chip_px}px | Year: {year}")
print(f"  Cache path: {out_tif.name}")

# Check if already cached
if out_tif.exists():
    print(f"\n✓ Using cached GeoTIFF (exists)")
    cache_time = time.time()
else:
    print(f"\n⏳ Downloading from Sentinel Hub API...")
    start = time.time()
    
    try:
        code, chip = landcover_at_point(
            client_id, client_secret,
            lat, lon,
            token_url=TOKEN_URL,
            process_url=PROCESS_URL,
            collection_id=COLLECTION_ID,
            year=year,
            buffer_m=buffer_m,
            chip_px=chip_px,
            save_path=str(out_tif),
        )
        
        elapsed = time.time() - start
        print(f"✓ Successfully saved GeoTIFF: {out_tif.name}")
        print(f"  Download time: {elapsed:.2f}s")
        print(f"  Center land cover code: {code}")
        print(f"  Chip shape: {chip.shape}")
    except Exception as e:
        print(f"✗ Error downloading: {e}")
        raise

print("\n" + "="*60)
print("PHASE 1 COMPLETE: Land cover cached")
print("="*60)
print(f"\nOutput file: {out_tif}")
print(f"\nNext: Run Phase 2 (phase2_batch_points.ipynb) to generate receiver points")


PHASE 1: DATA PREPARATION - Fetch Land Cover

Resolving Sentinel Hub credentials...
  ✓ Found credentials in config file

Fetching land cover for transmitter:
  Location: (9.345, -13.40694)
  Buffer: 11000m | Chip: 734px | Year: 2020
  Cache path: lcm10_9.345_-13.40694_2020_buf11000m_734px.tif

✓ Using cached GeoTIFF (exists)

PHASE 1 COMPLETE: Land cover cached

Output file: /Users/oz/Documents/mst_gis/data/intermediate/api_data/lcm10_9.345_-13.40694_2020_buf11000m_734px.tif

Next: Run Phase 2 (phase2_batch_points.ipynb) to generate receiver points


## Summary

**Phase 1 Complete**:
- ✓ Credentials resolved
- ✓ Land cover GeoTIFF cached locally
- ✓ Ready for Phase 2 (batch point generation)

**Output**: Cached GeoTIFF file ready for phase 2-4 processing